### Try 1

In [ ]:
import pyrealsense2 as rs
import cv2
import datetime 
import numpy as np
import matplotlib.pyplot as plt

xaxis = 200
yaxis = 20

pipe = rs.pipeline()
cfg = rs.config()

cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

pipe.start(cfg)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

while True:
    frame = pipe.wait_for_frames()

        # Align the frames
    aligned_frames = align.process(frame)

    #depth_frame = frames.get_depth_frame()
    #color_frame = frames.get_color_frame()

    # Get the aligned color and depth frames
    aligned_color_frame = aligned_frames.get_color_frame()
    aligned_depth_frame = aligned_frames.get_depth_frame()


    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())
    depth_cm = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.5), cv2.COLORMAP_JET)

    gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

    cv2.drawMarker(color_image, (xaxis, yaxis), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

    # Put images in the code
        # Get depth at coordinates (xaxis, yaxis)
    depth_value = depth_frame.get_distance(xaxis, yaxis)
    depth_text = f"Depth: {depth_value:.2f} meters"
    cv2.putText(color_image, depth_text, (xaxis+10, yaxis-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)


    cv2.imshow('rgb', color_image)
    cv2.imshow('depth', depth_cm)

    if cv2.waitKey(1) == ord('q'):
        break
    #plt.imshow(gray_image)
    #plt.scatter(xaxis, yaxis, color='red', marker='x')  # Mark the point (20, 20) with a red 'x'
    #plt.axis('off')  # Optional, to turn off the axis labels
    #plt.show()

    # Get depth at coordinates (xaxis, yaxis)
    depth_value = depth_frame.get_distance(xaxis, yaxis)
    print(f"Depth at ({xaxis}, {yaxis}): {depth_value} meters")

pipe.stop()


### Try 2

In [ ]:
import pyrealsense2 as rs
import cv2
import numpy as np

xaxis = 200
yaxis = 20

# define the screen size
width = 640
height = 480

# init. the camera pipeline
pipeline = rs.pipeline()
# camera settings
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)
# start the camera pipeline
pipeline.start(config)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# it is a good practice to wait a few frames before the main stream
for _ in range(10):
    pipeline.wait_for_frames()

try:
    while True:
        frames = pipeline.wait_for_frames()

        # Align the frames
        aligned_frames = align.process(frames)

        # Get the aligned color and depth frames
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue

        # Get BGR and Depth data, then create a depth colorized image
        color_image = np.asanyarray(aligned_color_frame.get_data())
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Draw a marker at (xaxis, yaxis) on the color image
        cv2.drawMarker(color_image, (xaxis, yaxis), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

        # Get depth at coordinates (xaxis, yaxis)
        depth_value = aligned_depth_frame.get_distance(xaxis, yaxis)
        depth_text = f"Depth: {depth_value:.2f} meters"
        cv2.putText(color_image, depth_text, (xaxis + 10, yaxis - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        # Stack the color image and depth colormap
        show_dual_win = np.hstack((color_image, depth_colormap))

        # Show the color image with depth information
        cv2.imshow('Dual Window', show_dual_win)

        pressed_key = cv2.waitKey(1) & 0xFF

        if pressed_key == ord('q'):
            # Quit
            break

        if pressed_key == ord('s'):
            # Save image or perform other actions
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


## Project 

### Imports

In [ ]:
import pyrealsense2 as rs
import cv2
import numpy as np



### Inittialize vars and settings

In [78]:
# Coordinates of the two points
x1, y1 = 200, 20
x2, y2 = 200, 200

# Define the screen size
width = 640
height = 480

# Initialize the camera pipeline
pipeline = rs.pipeline()

# Camera settings
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)

# Start the camera pipeline
pipeline.start(config)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# Wait for a few frames before the main stream
# This is to allow the camera time to adjust its brightness and other camera settings and stabilize
for _ in range(10):
    pipeline.wait_for_frames()

# Get depth scale for converting the depth values to meters
# Intrinsics are used to get the depth scale

profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()

try:
    while True:
        frames = pipeline.wait_for_frames()

        # Align the frames
        aligned_frames = align.process(frames)

        # Get the aligned color and depth frames
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue

        # Get BGR and Depth data, then create a depth colorized image
        color_image = np.asanyarray(aligned_color_frame.get_data())
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Draw markers at the two points on the color image
        cv2.drawMarker(color_image, (x1, y1), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
        cv2.drawMarker(color_image, (x2, y2), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

        # Get depth values at the two points
        depth_value1 = aligned_depth_frame.get_distance(x1, y1)
        depth_value2 = aligned_depth_frame.get_distance(x2, y2)

        # Convert from pixel coordinates to real world coordinates 
        point1 = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x1, y1], depth_value1)
        point2 = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x2, y2], depth_value2)

        # Calculate the euclidean distance between the two points with depth values as z coordinates
        diff = np.subtract(point1, point2)
        euclidean_distance = np.linalg.norm(diff)

        # Display the distance on the color image
        distance_text = f"Distance: {euclidean_distance:.2f} meters"
        cv2.putText(color_image, distance_text, (x1 + 10, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        # Trace a line between the two points
        cv2.line(color_image, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Stack the color image and depth colormap
        show_dual_win = np.hstack((color_image, depth_colormap))

        # Show the color image with depth information
        cv2.imshow('Dual Window', show_dual_win)

        pressed_key = cv2.waitKey(1) & 0xFF

        if pressed_key == ord('q'):
            # Quit
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()



### Now detection of 2 points that are not hardcoded and have the same color and the euclidean distance between them

In [ ]:
# Coordinates of the two points
x1, y1 = 200, 20
x2, y2 = 200, 200

# Define the screen size
width = 640
height = 480

# Initialize the camera pipeline
pipeline = rs.pipeline()

# Camera settings
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)

# Start the camera pipeline
pipeline.start(config)

# Create an align object
align_to = rs.stream.color
align = rs.align(align_to)

# Wait for a few frames before the main stream
# This is to allow the camera time to adjust its brightness and other camera settings and stabilize
for _ in range(10):
    pipeline.wait_for_frames()

# Get depth scale for converting the depth values to meters
# Intrinsics are used to get the depth scale

profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()

try:
    while True:
        frames = pipeline.wait_for_frames()

        # Align the frames
        aligned_frames = align.process(frames)

        # Get the aligned color and depth frames
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue

        # Get BGR and Depth data, then create a depth colorized image
        color_image = np.asanyarray(aligned_color_frame.get_data())
        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        # Draw markers at the two points on the color image
        cv2.drawMarker(color_image, (x1, y1), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
        cv2.drawMarker(color_image, (x2, y2), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

        # Get depth values at the two points
        depth_value1 = aligned_depth_frame.get_distance(x1, y1)
        depth_value2 = aligned_depth_frame.get_distance(x2, y2)

        # Convert from pixel coordinates to real world coordinates 
        point1 = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x1, y1], depth_value1)
        point2 = rs.rs2_deproject_pixel_to_point(depth_intrinsics, [x2, y2], depth_value2)

        # Calculate the euclidean distance between the two points with depth values as z coordinates
        diff = np.subtract(point1, point2)
        euclidean_distance = np.linalg.norm(diff)

        # Display the distance on the color image
        distance_text = f"Distance: {euclidean_distance:.2f} meters"
        cv2.putText(color_image, distance_text, (x1 + 10, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        # Trace a line between the two points
        cv2.line(color_image, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Stack the color image and depth colormap
        show_dual_win = np.hstack((color_image, depth_colormap))

        # Show the color image with depth information
        cv2.imshow('Dual Window', show_dual_win)

        pressed_key = cv2.waitKey(1) & 0xFF

        if pressed_key == ord('q'):
            # Quit
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()



## Day 2

### Code 1


In [67]:
# mean of last 10 frames

def stabilization_values(last_depth_values: list, depth_value: float) -> float:
    # Add the depth value to the list and remove the oldest one if the list has more than 10 items
    last_depth_values.append(depth_value)
    if len(last_depth_values) > 50:
        last_depth_values.pop(0)

    # Calculate the mean depth value
    mean_depth_value = sum(last_depth_values) / len(last_depth_values)

    return mean_depth_value, last_depth_values


In [ ]:
def depth():
    return 0

In [ ]:
def distance():
    return 0

#### Save data

In [ ]:
def save_data(frame_count: int, color_mtx, depth_mtx) -> None:
    # Save frames every 100 frames
    if frame_count % 100 == 0:
        np.save(f'color_{frame_count}.npy', color_mtx)
        np.save(f'depth_{frame_count}.npy', depth_mtx)

In [111]:
import pyrealsense2 as rs
import cv2
import numpy as np
import pickle
import time

xaxis = 320
yaxis = 240
width = 848
height = 480
scale_multiplier = 1 #0.91

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, width, height, rs.format.z16, 30)
config.enable_stream(rs.stream.color, width, height, rs.format.bgr8, 30)
pipeline.start(config)

align_to = rs.stream.color
align = rs.align(align_to)
last_frame_values_depth = []
display_point = None
display_text = ""

frame_count = 0

# Callback function for mouse click
# TODO: correct the mean depth value calculation when the mouse is clicked
def mouse_click(event, x, y, flags, param):
    global display_point, display_text, last_frame_values_depth, mean_depth_value
    if event == cv2.EVENT_LBUTTONDOWN:
        for i in range(2):
            depth_value = aligned_depth_frame.get_distance(x, y) * scale_multiplier
            display_point = (x, y)
            display_text = f"Distance: {depth_value:.2f} meters"
            time.sleep(1)

cv2.namedWindow('dual_win')
cv2.setMouseCallback('dual_win', mouse_click)

try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        aligned_color_frame = aligned_frames.get_color_frame()
        aligned_depth_frame = aligned_frames.get_depth_frame()

        if not aligned_depth_frame or not aligned_color_frame:
            continue

        color_mtx = np.asanyarray(aligned_color_frame.get_data())
        depth_mtx = np.asanyarray(aligned_depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_mtx, alpha=0.03), cv2.COLORMAP_JET)

        frame_count += 1

        #cv2.drawMarker(color_mtx, (xaxis, yaxis), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
        show_dual_win = np.hstack((color_mtx,depth_colormap))
        
        pressed_key = cv2.waitKey(1) & 0xFF
        
        if pressed_key == ord('q'):
            break
            
        if pressed_key == ord('s'):
            break

        # Display the cross marker and distance at the last clicked location
        if display_point is not None:
            cv2.drawMarker(show_dual_win, display_point, (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
            cv2.putText(show_dual_win, display_text, (display_point[0] + 10, display_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        
        mean_depth_value, last_frame_values_depth = stabilization_values(last_frame_values_depth, depth_value)

        cv2.imshow('dual_win', show_dual_win)

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


#### Load data

In [104]:
import numpy as np
import cv2
import os


frame_count = len(os.listdir()) // 2  # Divide by 2 because we have color and depth frames
xaxis = 320
yaxis = 240

try:
    for i in range(100, 500, 100):  # multiply frame_count by 100 because we saved frames every 100 frames
        color_frame = np.load(f'color_{i}.npy')
        depth_frame = np.load(f'depth_{i}.npy')

        depth_value = depth_frame[yaxis, xaxis]

        # Display the distance on the color image
        distance_text = f"Distance: {depth_value:.2f} meters"
        cv2.putText(color_frame, distance_text, (xaxis + 10, yaxis - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        cv2.imshow('color_frame', color_frame)

        if cv2.waitKey(66) & 0xFF == ord('q'):  # Delay of 66 ms for approx 15 fps
            break
finally:
    cv2.destroyAllWindows()
